In [23]:
import os
import cv2
import numpy as np
import shutil

In [24]:
def enhance_image(image):
    
    #normalizing
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0  


    #deblur
    def richardson_lucy_rgb(image, psf, num_iterations=10):

        if image.dtype == np.uint8:
            image = image.astype(np.float32) / 255.0 
        psf = psf / np.sum(psf)  
        psf_flip = cv2.flip(psf, -1)  

        channels = cv2.split(image)  
        deblurred_channels = []

        for ch in channels:
            estimate = ch.copy()

            for _ in range(num_iterations):
                blurred_estimate = cv2.filter2D(estimate, -1, psf)
                ratio = np.clip(ch / (blurred_estimate + 1e-8), 0, 2)  
                correction = cv2.filter2D(ratio, -1, psf_flip)
                estimate *= correction  

            deblurred_channels.append(np.clip(estimate, 0, 1))

        deblurred_image = cv2.merge(deblurred_channels)  

        return deblurred_image
    
    psf = np.array([[1, 4, 7, 4, 1],
                [4, 16, 26, 16, 4],
                [7, 26, 41, 26, 7],
                [4, 16, 26, 16, 4],
                [1, 4, 7, 4, 1]], dtype=np.float32)
    
    deblurred_image = richardson_lucy_rgb(image, psf, num_iterations=15)
    
    #histogram
    def apply_gamma_correction(image, gamma=1.5):
        image = (image * 255).astype(np.uint8) 

        inv_gamma = 1.0 / gamma
        table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(256)]).astype("uint8")


        corrected_image = cv2.LUT(image, table).astype(np.float32) / 255.0
        return corrected_image
    
    final_image = apply_gamma_correction(deblurred_image)

    final_image = cv2.cvtColor(final_image,cv2.COLOR_RGB2BGR)


    return final_image



In [25]:
def Setpath():
    original_dataset = r"F:\Project\OGDATASET\Test"
    enhanced_dataset = r"F:\Project\Dataset_enhanced\Test"  # Change this to your dataset path

    # original_dataset = os.path.join(original_dataset,class_id)
    # enhanced_dataset = os.path.join(enhanced_dataset,class_id)

    # print("original_dataset : " , original_dataset , " and enhanced dataset : " , enhanced_dataset)

    if os.path.exists(enhanced_dataset):
        shutil.rmtree(enhanced_dataset)  # Remove existing enhanced dataset
    shutil.copytree(original_dataset, enhanced_dataset, dirs_exist_ok=True)  

    # Process all images
    for root, _, files in os.walk(original_dataset):
        for file in files:
            if file.endswith((".png", ".jpg", ".jpeg")):
                original_path = os.path.join(root, file)
                enhanced_path = original_path.replace(original_dataset, enhanced_dataset)

                # Read and enhance
                image = cv2.imread(original_path)
                finalImg = enhance_image(image)
                finalImg = (finalImg * 255).astype(np.uint8)


                # # Save enhanced image
                cv2.imwrite(enhanced_path, finalImg)
                # break

    print("✅ Enhanced dataset saved in:", enhanced_dataset)

In [26]:
# folders = ['test','train','val']

# for i in range(0,43):
#     Setpath(str(i))
Setpath()

✅ Enhanced dataset saved in: F:\Project\Dataset_enhanced\Test


In [ ]:
# if os.path.exists(enhanced_dataset):
#     shutil.rmtree(enhanced_dataset)  # Remove existing enhanced dataset
# shutil.copytree(original_dataset, enhanced_dataset, dirs_exist_ok=True)  

# # Process all images
# for root, _, files in os.walk(original_dataset):
#     for file in files:
#         if file.endswith((".png", ".jpg", ".jpeg")):
#             original_path = os.path.join(root, file)
#             enhanced_path = original_path.replace(original_dataset, enhanced_dataset)

#             # Read and enhance
#             image = cv2.imread(original_path)
#             finalImg = enhance_image(image)
#             finalImg = (finalImg * 255).astype(np.uint8)


#             # # Save enhanced image
#             cv2.imwrite(enhanced_path, finalImg)
#             # break

# print("✅ Enhanced dataset saved in:", enhanced_dataset)

✅ Enhanced dataset saved in: F:\Project\Dataset2_enhanced\Train\0
